In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os
import requests
from gzip import GzipFile

# Descargar el dataset MNIST
def download_mnist_files():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    files = ["train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz",
             "t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz"]
    for file in files:
        if not os.path.isfile(file) or os.path.getsize(file) == 0:  # Verificar si el archivo no existe o está vacío
            print(f"Descargando {file}...")
            resp = requests.get(base_url + file)
            if resp.status_code == 200 and 'application/x-gzip' in resp.headers.get('Content-Type', ''):
                with open(file, 'wb') as f:
                    f.write(resp.content)
                if os.path.getsize(file) > 0:  # Verificar que el archivo no esté vacío después de la descarga
                    print(f"{file} descargado correctamente.")
                else:
                    print(f"Error: {file} se descargó pero está vacío.")
            else:
                print(f"Error al descargar {file}. Respuesta: {resp.status_code}, Tipo: {resp.headers.get('Content-Type')}")
        else:
            print(f"{file} ya existe y no está vacío.")

# Cargar y preparar los datos
def load_mnist_data():
    download_mnist_files()
    def read_images(filename):
        with GzipFile(filename, 'rb') as f:
            # Omitir la cabecera
            f.read(16)
            # Leer el resto de los datos
            data = np.frombuffer(f.read(), np.uint8, offset=0)
        return data.reshape(-1, 28, 28).astype(np.float32) / 255  # Normalizar

    def read_labels(filename):
        with GzipFile(filename, 'rb') as f:
            f.read(8)  # Omitir la cabecera
            data = np.frombuffer(f.read(), np.uint8, offset=0)
        return data

    # Cargar los datos
    x_train = read_images("train-images-idx3-ubyte.gz")
    y_train = read_labels("train-labels-idx1-ubyte.gz")
    x_test = read_images("t10k-images-idx3-ubyte.gz")
    y_test = read_labels("t10k-labels-idx1-ubyte.gz")

    # Convertir a tensores de PyTorch
    x_train, y_train, x_test, y_test = map(
        torch.tensor, (x_train, y_train, x_test, y_test)
    )
    return x_train, y_train, x_test, y_test

# Definir la red neuronal
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# Entrenamiento
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Pérdida: {loss.item():.6f}")

# Evaluación
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nConjunto de prueba: Pérdida promedio: {test_loss:.4f}, Precisión: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

# Cargar datos
x_train, y_train, x_test, y_test = load_mnist_data()
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

# Crear DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Configurar dispositivo y modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)

# Optimizador
optimizer = optim.Adam(model.parameters())

# Ciclo de entrenamiento
for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

BadGzipFile: Not a gzipped file (b'<!')